In [ ]:
import pandas as pd
import re
from transformers import AutoTokenizer

tokenizer_name = "dbmdz/bert-base-turkish-uncased"
LOWERCASE = True

try:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    print(f"BERTurk tokenizer ({tokenizer_name}) başarıyla yüklendi.")
except OSError:
    print(f"HATA: BERTurk tokenizer ({tokenizer_name}) bulunamadı veya indirilemedi.")
    print("İnternet bağlantınızı kontrol edin veya model adının doğru olduğundan emin olun.")
    raise # Hata oluştuysa devam etme

BERTurk tokenizer (dbmdz/bert-base-turkish-uncased) başarıyla yüklendi.


In [36]:
df = pd.read_csv("/content/tum_yorumlar.csv")

In [37]:
df.head()

,yorum
0,Filim başka bir bakış açısıyla muhteşem ve son...
1,"Sonuç olarak gerçekten güzel bir film , tavsiy..."
2,"İnsana aşırı pozitif hissettiren, günlük hayat..."
3,uzun zaman sonra izlediğim en güzel filmlerden...
4,Uzun süredir keyif alarak izlediğim bir komedi...


In [38]:
#df.drop(columns=["point"], inplace=True)
df.rename(columns={"yorum": "comment"}, inplace=True)
df["comment"]=df["comment"].str.replace('\n', "")
df['comment'] = df['comment'].str.strip()
df['comment'] = df['comment'].str.replace('"', '')

In [39]:
df.head()

,comment
0,Filim başka bir bakış açısıyla muhteşem ve son...
1,"Sonuç olarak gerçekten güzel bir film , tavsiy..."
2,"İnsana aşırı pozitif hissettiren, günlük hayat..."
3,uzun zaman sonra izlediğim en güzel filmlerden...
4,Uzun süredir keyif alarak izlediğim bir komedi...


In [40]:
chars_to_normalize_map_from = "ÂâÎîÛûÊêÄäËëÏïÖöÜü"
chars_to_normalize_map_to   = "AaIiUuEeAaEeIiOoUu"

translation_table = str.maketrans(chars_to_normalize_map_from, chars_to_normalize_map_to)

def preprocess_for_berturk(text):
  """
  Türkçe metinleri BERTurk modeline uygun hale getirmek için temizleyceğiz
  """
  if not isinstance(text, str) or not text.strip():
    return ""

  if LOWERCASE:
    text = text.lower()

  # ---- Karakter Normalizasyonu ---
  text = text.translate(translation_table)

  # ---- URL'lerin kaldırılması ---
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

  # ---- @mention'ların kaldırılması ---
  text = re.sub(r'\@\w+', '', text)

  # ---- hashtag isaretinin kaldrılması ---
  text = text.replace("#", "")

  text = re.sub(r'[^a-z0-9ığüşöç\s.,!?:;]', '', text)

  text = re.sub(r'[.,!?:;\(\)\[\]\{\}]', '', text)

  # ---- Sayılarin kaldırilmasi ---
  text = re.sub(r'\d+', '', text)

  # ---- Ekstra boşlukların kaldırılması  ---
  text = re.sub(r'\s+', ' ', text).strip()

  return text


print("Karakter normalizasyonu içeren ön işleme fonksiyonu 'preprocess_for_berturk_normalized' tanımlandı.")

Karakter normalizasyonu içeren ön işleme fonksiyonu 'preprocess_for_berturk_normalized' tanımlandı.


In [41]:
print("\nBERTurk için ön işleme (normalizasyonlu) BAŞLIYOR...")
if 'comment' not in df.columns:
  raise ValueError("DataFrame'de 'comment' sütunu bulunamadı.")

df['processed_comment'] = df['comment'].astype(str).apply(preprocess_for_berturk)
print("Ön işleme tamamlandı.")


BERTurk için ön işleme (normalizasyonlu) BAŞLIYOR...
Ön işleme tamamlandı.


In [42]:
df.head()

,comment,processed_comment
0,Filim başka bir bakış açısıyla muhteşem ve son...,filim başka bir bakış açısıyla muhteşem ve son...
1,"Sonuç olarak gerçekten güzel bir film , tavsiy...",sonuç olarak gerçekten guzel bir film tavsiye ...
2,"İnsana aşırı pozitif hissettiren, günlük hayat...",insana aşırı pozitif hissettiren gunluk hayatı...
3,uzun zaman sonra izlediğim en güzel filmlerden...,uzun zaman sonra izlediğim en guzel filmlerden...
4,Uzun süredir keyif alarak izlediğim bir komedi...,uzun suredir keyif alarak izlediğim bir komedi...


In [43]:
df_processed = df.drop(columns=["comment"])
df_processed.rename(columns={"processed_comment": "comment"}, inplace=True)

In [44]:
df_processed.head()

,comment
0,filim başka bir bakış açısıyla muhteşem ve son...
1,sonuç olarak gerçekten guzel bir film tavsiye ...
2,insana aşırı pozitif hissettiren gunluk hayatı...
3,uzun zaman sonra izlediğim en guzel filmlerden...
4,uzun suredir keyif alarak izlediğim bir komedi...


In [45]:
df_processed.to_csv("processed_beyazperde_review_dataset.csv", index=False)

In [46]:
from google.colab import files
files.download('processed_beyazperde_review_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>